<a href="https://colab.research.google.com/github/Jass1313c/comp215/blob/main/labs/lab06_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 6
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * create a Watts-Strogatz graph both from scratch and from the Networkx module
  * measure the average clustering coefficient and path length of a network
  * visualize summative data of a graph


During this lab, you will be introduced to the following:
  * numpy arrays
  * local file i/o in Google Colab

(this lab is based on workbooks provided in Allen Downey's 'Think Complexity')

## Social Networking

This lab uses graphs to explore social networks using Facebook data.  In this lab you will create a simulated model of the network using a Watts-Strogatz graph and compare some properties of the simulated network and the real Facebook network.

In [ ]:
# put your imports here

In [1]:
import numpy as np
import networkx as nx
import random
from collections import deque

## Get the Facebook Data

Stanford Network Analysis Platform (SNAP) is a general purpose network analysis and graph mining library.  In previous labs, we have used APIs to access data.  For this lab, we will copy the data to a local file.  Download the ```facebook_combined.txt.gz``` file from [SNAP](https://snap.stanford.edu/data/egonets-Facebook.html), unzip it, and copy ```facebook_combined.txt``` to the ```Files``` folder in Colab.  

Look at the content of the file and read the SNAP webpage to understand what the data represents before moving on to the rest of the lab.

## Exercise 1: Make Facebook Graph

Write a function that reads the file, one edge per line, specified by the two integer node IDs given in each line of the file and returns a ```networkx``` graph representing the data.  You can do this with Python's built-in file handling, or you could use ```numpy```'s ```loadtxt``` function.  Write a unit test to check that the network has 4039 nodes and 88234 edges (as given in the Dataset Statistics on the SNAP site) and draw the Facebook network (this takes about a minute).


In [8]:
# Ex 1 code here
def facebook_graph():
  graph = nx.Graph()

  with open('facebook_combined.txt', 'r') as file:
    for line in file:
      node1, node2 = map(int, line.strip().split())
      graph.add_edge(node1, node2)

  return graph

def test_facebook_graph():

  graph = facebook_graph()

  assert graph.number_of_nodes() == 4039
  assert graph.number_of_edges() == 88234

  print("Test passed!")

test_facebook_graph()

Test passed!


## Exercise 2: Clustering Coefficients

With larger graphs, it can take a long time to compute clustering coefficients and path lengths. We can estimate them by sampling without much loss of accuracy if the sample size is large enough.  Write a function that calculates the average clustering coeffient for a random subset of a N nodes in a network.  You may use the ```node_clustering``` and ```all_pairs``` functions from Chapter 5 of the textbook.  You may also use the ```numpy``` module to calculate the mean; note that there is a ```nanmean``` function.

Check that your clustering coeffients function gives a similar answer to the ```networkx``` ```average_clustering``` function.


In [20]:
# Ex 2 code here
import random

def clustering_coefficient(G, N=10):
    nodes = random.sample(list(G.nodes()), N)

    clustering_coeffs = [nx.clustering(G, node) for node in nodes]
    return np.nanmean(clustering_coeffs)

G = facebook_graph()

avg_clustering = clustering_coefficient(G, N=100)
print("Average Clustering Coefficient (custom):",avg_clustering)
print(clustering_coefficient(G, N=10))

avg_clustering_nx = nx.average_clustering(G)
print("Average Clustering Coefficient (networkx):",avg_clustering_nx)

print("Results are similar:", np.isclose(avg_clustering, avg_clustering_nx, atol = 0.05))

Average Clustering Coefficient (custom): 0.5944692802939191
0.5434325655129111
Average Clustering Coefficient (networkx): 0.6055467186200876
Results are similar: True


## Exercise 3: Average Shortest Path Length

Write a function that calculates the average shortest path length for all pairs of nodes in a network.  You may use the ```shortest_path_dijkstra``` function from Chapter 5 of the textbook.  Using that function, it took my algorithm about 2 minutes to find the average shortest path over all pairs of nodes.


Check that your average shortest path length function gives a similar answer to the ```networkx``` ```average_shortest_path_length``` function.


In [29]:
# Ex 3 code here
def avg_shortest_path_length(G):

  total_path_length = 0
  num_paths = 0

  for node1 in G.nodes():
    for node2 in G.nodes():
      if node1 != node2:
        path_length = nx.shortest_path_length(G, source=node1, target=node2)
        total_path_length += path_length
        num_paths += 1

    average_path_length = total_path_length / num_paths
    return average_path_length

G = facebook_graph()

custom_avg_path_length = avg_shortest_path_length(G)
nx_avg_path_length = nx.average_shortest_path_length(G)


print("Average Shortest Path Length (custom):", custom_avg_path_length)
print("Average Shortest Path Length (networkx):", nx_avg_path_length)

print("Results are similar:", np.isclose(custom_avg_path_length, nx_avg_path_length))


Average Shortest Path Length (custom): 2.83011391778108
Average Shortest Path Length (networkx): 3.6925068496963913
Results are similar: False


Here is a function from the textbook that takes a sample of path lengths to estimate the average shortest path length.  You may use this in the rest of the lab so that you don't need to wait for the whole full averaging algorithms above to run.

In [22]:

def sample_path_lengths(G, nodes=None, trials=100):
    """Choose random pairs of nodes and compute the path length between them.
    G: Graph
    N: number of pairs to choose
    returns: list of path lengths
    """
    if nodes is None:
        nodes = list(G)
    else:
        nodes = list(nodes)

    pairs = np.random.choice(nodes, (trials, 2))
    lengths = [nx.shortest_path_length(G, *pair)
               for pair in pairs]
    return lengths

def estimate_path_length(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

In the exercises above, you should have found that the Facebook network has an average clustering coefficient around 0.6 and an average shortest path length of around 3.7. Note that this corresponds to a 'degree of separation' of less than 6.   

## Exercise 4: WS Graph

Construct a WS graph with the same number of nodes and average degree as the Facebook network using the ```make_ws_graph``` function from Chapter 5.  Find the value of p (probability of rewire) that reproduces a clustering coefficient and average shortest path length of the Facebook network.  (Note that there is a ```nx.watts_strogatz_graph``` that you may use after you have demonstrated that you can create a WS graph using the functions from Chapter 5.).

What could this value of p tell you about the actual social network that this Facebook data represents?  (Think about what p means in the model and what that would represent in the data.)

In [35]:
# Ex 4 code here
def WS_graph(n, k, p):

  ws = nx.Graph()
  nodes = range(n)
  ws.add_nodes_from(nodes)

  for i in nodes:
    for j in range(1, k // 2 + 1):
      target = (i + j) % n
      ws.add_edge(i, target)

  for i in nodes:
    for j in range(1, k // 2 + 1):
      target = (i - j) % n
      ws.add_edge(i, target)

  for i in nodes:
    for j in nodes:
      if random.random() < p:
        neighbors = (i + j) % n
        new_neighbor = random.choice(nodes)
        while new_neighbor == i or ws.has_edge(i, new_neighbor):
          new_neighbor = random.choice(nodes)


        ws.add_edge(i, new_neighbor)

  return ws

facebook_network = facebook_graph()
facebook_avg_degree = np.mean([d for _, d in facebook_network.degree()])
facebook_clustering_coeff = nx.average_clustering(facebook_network)
facebook_avg_path_length = estimate_path_length(facebook_network)

print("Facebook Network - Average Degree:", facebook_avg_degree)
print("Facebook Network - Clustering Coefficient:", facebook_clustering_coeff)
print("Facebook Network - Average Path Length:", facebook_avg_path_length)

n = facebook_network.number_of_nodes()
k = int(round(facebook_avg_degree))

best_p = 0
min_diff = float('inf')

for p in np.linspace(0, 1, 20):
  ws_graph = WS_graph(n, k, p)
  ws_clustering_coeff = nx.average_clustering(ws_graph)
  ws_avg_path_length = estimate_path_length(ws_graph)

  diff = abs(ws_clustering_coeff - facebook_clustering_coeff) + abs(ws_avg_path_length - facebook_avg_path_length)

  if diff < min_diff:
    min_diff = diff
    best_p = p

print("Optimal Rewiring Probability (p):", best_p)

Facebook Network - Average Degree: 43.69101262688784
Facebook Network - Clustering Coefficient: 0.6055467186200876
Facebook Network - Average Path Length: 3.7


KeyboardInterrupt: 